In [ ]:
''' Plotting script'''
import pandas as pd
import seaborn as sns
import numpy as np
from matplotlib import pyplot as plt
import matplotlib.gridspec as gridspec
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams.update({'pdf.fonttype': 42})

In [ ]:
''' Helper functions to construct dataframes '''
# Create dataframe from csv files of different algorithms.
def create_dataframe(prefix:str, var_name:str, vars:list, algos:dict):
    # Declare the data frame.
    df = pd.DataFrame(columns=['Algorithm', var_name])

    # Read data into dataframe.
    for var in vars:
        for algo in algos.keys():
            try:
                dir = "{}/{}={}/".format(prefix, var_name, var)
                value = pd.read_csv("{}/{}-performance.csv".format(dir, algo), delimiter=",", header=None)
                df = pd.concat([df, value.iloc[-1].T*100], ignore_index=True)
                df.loc[len(df)-len(value.iloc[-1].T):,'Algorithm'] = algos[algo]
                df.loc[len(df)-len(value.iloc[-1].T):, var_name] = var
            except:
                pass
    df = df.rename(columns={0: 'IRC (%)'})
    return df

def read_default_value(dir):
    df = create_dataframe(dir, 'intensity', [0.5], {'Central': "Cen-MCTS"})
    return np.array(df[(df['Algorithm']=='Cen-MCTS') & (df['intensity']==0.5)]['IRC (%)'])

def create_improvement_dataframe(og_df:pd.DataFrame, var_name:str, vars:list, algos:dict, dir:str=''):
    df = pd.DataFrame(columns=['Algorithm', var_name])
    if len(dir) > 0:
        bench_mark = read_default_value(dir)
    for algo in algos:
        if algo != 'Central':
            for var in vars:
                if len(dir) == 0:
                    bench_mark = np.array(og_df[(og_df['Algorithm'] == 'Cen-MCTS') & (og_df[var_name] == var)]['IRC (%)'])
                compared_method = np.array(og_df[(og_df['Algorithm'] == algos[algo]) & (og_df[var_name] == var)]['IRC (%)'])
                num_sample = min(len(bench_mark), len(compared_method))
                value = pd.DataFrame(compared_method[:num_sample] - bench_mark[:num_sample])
                df = pd.concat([df, value], ignore_index=True)
                df.loc[len(df)-len(value):,'Algorithm'] = algos[algo]
                df.loc[len(df)-len(value):, var_name] = var
    df = df.rename(columns={0: 'Improvement Over Cen-MCTS (%)'})
    return df

In [ ]:
'''Message Missed'''
# Set up the figure.
f, axes = plt.subplots(1, 3, figsize=(30, 6))
sns.set_theme(context='paper', style='whitegrid', palette='deep', font='Times New Roman', font_scale=2.5, color_codes=True, rc=None)
threshold = [0, 1, 5, 10, 20]
ax = dict()
df_threshold = dict()
text = ['a', 'b', 'c']

# Create dataframe DC.
algos = {"RM": "A-MCTS"}
dir = "../../../INFOCOM-24/Data/9_action"
df_threshold[0] = create_dataframe(dir, "threshold", threshold, algos)

# Create dataframe ORI full comm.
algos = {"A": "A-MCTS"}
dir = "../../Data/full_comm"
df_threshold[1] = create_dataframe(dir, "threshold", threshold, algos)

# Create dataframe ORI real comm.
dir = "../../Data/real_comm"
df_threshold[2] = create_dataframe(dir, "threshold", threshold, algos)

# Plot the average score with 95% confidence interval.
for i in range(3):
    ax[i] = sns.lineplot(data=df_threshold[i], x="threshold", y='IRC (%)', hue='Algorithm', style='Algorithm', marker='^', palette=['orange'], linewidth=3, errorbar=None, markersize=12, ax=axes[i])
    ax[i] = sns.boxplot(data=df_threshold[i], x="threshold", y='IRC (%)', fill=False, color='r', whis=(5, 95), order=np.arange(21), ax=axes[i])
    ax[i].set(xlabel="Allowed Inter-Agent Communication Loss\n ({})".format(text[i]))
    ax[i].lines[0].set_linestyle("--")
    ax[i].set_ylim([40, 90])
    ax[i].set_xticks([i for i in threshold])
    ax[i].autoscale(enable=True, axis='x', tight=False)
    ax[i].get_legend().remove()

# Save figure to pdf.
# plt.savefig("../../Figure/tolerance.pdf", format="pdf", bbox_inches="tight")

In [ ]:
'''3  cases'''
# Set up the figure.
f = plt.figure(figsize=(30, 27))
gs = gridspec.GridSpec(3, 6)
sns.set_theme(context='paper', style='white', palette='deep', font='Times New Roman', font_scale=2.5, color_codes=True, rc=None)
ax = dict()
color = ['r', 'darkblue', 'skyblue', 'b', 'coral']
marker = ['^', 'D', 'o', 's', 'v']

dir = "../../../INFOCOM-24/Data/9_action"
algos = {"RM": "A-MCTS", "Central": "Cen-MCTS", "Dec": "Dec-MCTS", "Global": "Global-MCTS", "Reset": "Reset-MCTS", "Greedy": "Greedy-MCTS"}
# Create dataframe intensity.
intensity = [0.1, 0.3, 0.5, 0.7, 0.9]
df_intensity = create_dataframe(dir, "intensity", intensity, algos)
df_intensity = create_improvement_dataframe(df_intensity, 'intensity', intensity, algos)
df_intensity['intensity'] = df_intensity['intensity']*100
# Plot the average score with 95% confidence interval.
ax[0] = plt.subplot(gs[0, 0:2])
ax_1 = sns.lineplot(data=df_intensity, x="intensity", y='Improvement Over Cen-MCTS (%)', hue='Algorithm', style='Algorithm', err_style='bars', errorbar='ci', err_kws={'elinewidth': 2, 'capsize': 8, 'capthick': 2}, linewidth=5, markersize=20, palette=color, markers=marker, ax=ax[0])
ax_1.set(xlabel="Failures Intensity (%)\n (a)")
ax_1.set_xticks([i*100 for i in intensity])

# Create dataframe planning time.
planning = [100, 250, 500, 750, 1000]
df_planning = create_dataframe(dir, "planning", planning, algos)
df_planning = create_improvement_dataframe(df_planning, 'planning', planning, algos)
# Plot the average score with 95% confidence interval.
ax[1] = plt.subplot(gs[1, 0:2])
ax_2 = sns.lineplot(data=df_planning, x="planning", y='Improvement Over Cen-MCTS (%)', hue='Algorithm', style='Algorithm', err_style='bars', errorbar='ci', err_kws={'elinewidth': 2, 'capsize': 8, 'capthick': 2}, linewidth=5, markersize=20, palette=color, markers=marker, ax=ax[1])
ax_2.set(xlabel="Planning Time (iterations) \n (d)")
ax_2.set_xticks([i for i in planning])

# Create dataframe component exchanged.
algos = {"RM": "A-MCTS", "Dec": "Dec-MCTS", "Global": "Global-MCTS", "Reset": "Reset-MCTS", "Greedy": "Greedy-MCTS"}
comp = ['5', '10', '15', '20', '25']
df_comp = create_dataframe(dir, "comp", comp, algos)
df_comp = create_improvement_dataframe(df_comp, 'comp', comp, algos, dir)
# Plot the average score with 95% confidence interval.
ax[2] = plt.subplot(gs[2, 0:2])
ax_3 = sns.lineplot(data=df_comp, x="comp", y='Improvement Over Cen-MCTS (%)', hue='Algorithm', style='Algorithm', err_style='bars', errorbar='ci', err_kws={'elinewidth': 2, 'capsize': 8, 'capthick': 2}, linewidth=5, markersize=20, palette=color, markers=marker, ax=ax[2])
ax_3.set(xlabel="Number of Exchanged Components \n (g)")


dir = "../../Data/full_comm"
algos = {"A": "A-MCTS", "Central": "Cen-MCTS", "Dec": "Dec-MCTS", "Global": "Global-MCTS", "Reset": "Reset-MCTS", "Greedy": "Greedy-MCTS"}
# Create dataframe intensity.
intensity = [0.1, 0.3, 0.5, 0.7, 0.9]
df_intensity = create_dataframe(dir, "intensity", intensity, algos)
df_intensity = create_improvement_dataframe(df_intensity, 'intensity', intensity, algos)
df_intensity['intensity'] = df_intensity['intensity']*100
# Plot the average score with 95% confidence interval.
ax[3] = plt.subplot(gs[0, 2:4])
ax_4 = sns.lineplot(data=df_intensity, x="intensity", y='Improvement Over Cen-MCTS (%)', hue='Algorithm', style='Algorithm', err_style='bars', errorbar='ci', err_kws={'elinewidth': 2, 'capsize': 8, 'capthick': 2}, linewidth=5, markersize=20, palette=color, markers=marker, ax=ax[3])
ax_4.set(xlabel="Failures Intensity (%)\n (b)")
ax_4.set_xticks([i*100 for i in intensity])

# Create dataframe planning time.
planning = [40, 60, 80, 100, 120]
df_planning = create_dataframe(dir, "planning", planning, algos)
df_planning = create_improvement_dataframe(df_planning, 'planning', planning, algos)
# Plot the average score with 95% confidence interval.
ax[4] = plt.subplot(gs[1, 2:4])
ax_5 = sns.lineplot(data=df_planning, x="planning", y='Improvement Over Cen-MCTS (%)', hue='Algorithm', style='Algorithm', err_style='bars', errorbar='ci', err_kws={'elinewidth': 2, 'capsize': 8, 'capthick': 2}, linewidth=5, markersize=20, palette=color, markers=marker, ax=ax[4])
ax_5.set(xlabel="Planning Time (seconds) \n (e)")
ax_5.set_xticks([i for i in planning])

# Create dataframe component exchanged.
algos = {"A": "A-MCTS", "Dec": "Dec-MCTS", "Global": "Global-MCTS", "Reset": "Reset-MCTS", "Greedy": "Greedy-MCTS"}
comp = ['5', '10', '15', '20', '25']
df_comp = create_dataframe(dir, "comp", comp, algos)
df_comp = create_improvement_dataframe(df_comp, 'comp', comp, algos, dir)
# Plot the average score with 95% confidence interval.
ax[5] = plt.subplot(gs[2, 2:4])
ax_6 = sns.lineplot(data=df_comp, x="comp", y='Improvement Over Cen-MCTS (%)', hue='Algorithm', style='Algorithm', err_style='bars', errorbar='ci', err_kws={'elinewidth': 2, 'capsize': 8, 'capthick': 2}, linewidth=5, markersize=20, palette=color, markers=marker, ax=ax[5])
ax_6.set(xlabel="Number of Exchanged Components \n (h)")


dir = "../../Data/real_comm"
algos = {"A": "A-MCTS", "Central": "Cen-MCTS", "Dec": "Dec-MCTS", "Global": "Global-MCTS", "Reset": "Reset-MCTS", "Greedy": "Greedy-MCTS"}
# Create dataframe intensity.
df_intensity = create_dataframe(dir, "intensity", intensity, algos)
df_intensity = create_improvement_dataframe(df_intensity, 'intensity', intensity, algos)
df_intensity['intensity'] = df_intensity['intensity']*100
# Plot the average score with 95% confidence interval.
ax[6] = plt.subplot(gs[0, 4:6])
ax_7= sns.lineplot(data=df_intensity, x="intensity", y='Improvement Over Cen-MCTS (%)', hue='Algorithm', style='Algorithm', err_style='bars', errorbar='ci', err_kws={'elinewidth': 2, 'capsize': 8, 'capthick': 2}, linewidth=5, markersize=20, palette=color, markers=marker, ax=ax[6])
ax_7.set(xlabel="Failures Intensity (%)\n (c)")
ax_7.set_xticks([i*100 for i in intensity])

# Create dataframe planning time.
planning = [40, 60, 80, 100, 120]
df_planning = create_dataframe(dir, "planning", planning, algos)
df_planning = create_improvement_dataframe(df_planning, 'planning', planning, algos)
# Plot the average score with 95% confidence interval.
ax[7] = plt.subplot(gs[1, 4:6])
ax_8 = sns.lineplot(data=df_planning, x="planning", y='Improvement Over Cen-MCTS (%)', hue='Algorithm', style='Algorithm', err_style='bars', errorbar='ci', err_kws={'elinewidth': 2, 'capsize': 8, 'capthick': 2}, linewidth=5, markersize=20, palette=color, markers=marker, ax=ax[7])
ax_8.set(xlabel="Planning Time (seconds) \n (f)")
ax_8.set_xticks([i for i in planning])

# Create dataframe component exchanged.
algos = {"A": "A-MCTS", "Dec": "Dec-MCTS", "Global": "Global-MCTS", "Reset": "Reset-MCTS", "Greedy": "Greedy-MCTS"}
df_comp = create_dataframe(dir, "comp", comp, algos)
df_comp = create_improvement_dataframe(df_comp, 'comp', comp, algos, dir)
# Plot the average score with 95% confidence interval.
ax[8] = plt.subplot(gs[2, 4:6])
ax_9 = sns.lineplot(data=df_comp, x="comp", y='Improvement Over Cen-MCTS (%)', hue='Algorithm', style='Algorithm', err_style='bars', errorbar='ci', err_kws={'elinewidth': 2, 'capsize': 8, 'capthick': 2}, linewidth=5, markersize=20, palette=color, markers=marker, ax=ax[8])
ax_9.set(xlabel="Number of Exchanged Components \n (i)")

f.subplots_adjust(wspace=.5)
for i in range(9):
    ax[i].set_ylim([-15, 30])
    ax[i].margins(x=0.02)
    ax[i].axhline(y=0, linewidth=3, color='orange', ls='--')
    try:
        ax[i].get_legend().remove()
    except:
        pass
handles, labels = ax[0].get_legend_handles_labels()
leg = f.legend(handles, labels, markerscale=1, fontsize=25, loc='upper center', bbox_to_anchor=(0.5, 0.93), ncols=5)
for line in leg.get_lines():
    line.set_linewidth(5.0)

# Save figure to pdf.
# plt.savefig("../../Figure/full_failure_planning_comp.pdf", format="pdf", bbox_inches="tight")

In [ ]:
'''Budget - Agent - Reward 3 cases'''
# Set up the figure.
f = plt.figure(figsize=(30, 27))
gs = gridspec.GridSpec(3, 6)
sns.set_theme(context='paper', style='white', palette='deep', font='Times New Roman', font_scale=2.5, color_codes=True, rc=None)
ax = dict()
color = ['r', 'darkblue', 'skyblue', 'b', 'coral']
marker = ['^', 'D', 'o', 's', 'v']

algos = {"RM": "A-MCTS", "Central": "Cen-MCTS", "Dec": "Dec-MCTS", "Global": "Global-MCTS", "Reset": "Reset-MCTS", "Greedy": "Greedy-MCTS"}
dir = "../../../INFOCOM-24/Data/9_action"
# Create dataframe action.
action = ['7', '9', '11', '13', '15']
df_action = create_dataframe(dir, "action", action, algos)
df_action = create_improvement_dataframe(df_action, 'action', action, algos)
# Plot the average score with 95% confidence interval.
ax[0] = plt.subplot(gs[0, 0:2])
ax_1 = sns.lineplot(data=df_action, x="action", y='Improvement Over Cen-MCTS (%)', hue='Algorithm', style='Algorithm', err_style='bars', errorbar='ci', err_kws={'elinewidth': 2, 'capsize': 8, 'capthick': 2}, linewidth=5, markersize=20, palette=color, markers=marker, ax=ax[0])
ax_1.set(xlabel="Travel Budget (Number of Actions) \n (a)")

# Create dataframe agent.
agent = ['10', '15', '20', '25', '30']
df_agent = create_dataframe(dir, "agent", agent, algos)
df_agent = create_improvement_dataframe(df_agent, 'agent', agent, algos)
# Plot the average score with 95% confidence interval.
ax[3] = plt.subplot(gs[1, 0:2])
ax_2 = sns.lineplot(data=df_agent, x="agent", y='Improvement Over Cen-MCTS (%)', hue='Algorithm', style='Algorithm', err_style='bars', errorbar='ci', err_kws={'elinewidth': 2, 'capsize': 8, 'capthick': 2}, linewidth=5, markersize=20, palette=color, markers=marker, ax=ax[3])
ax_2.set(xlabel="Number of Agents \n (d)")

# Create dataframe rewards.
reward = ['100', '200', '300', '400', '500']
df_reward = create_dataframe(dir, "reward", reward, algos)
df_reward = create_improvement_dataframe(df_reward, 'reward', reward, algos)
# Plot the average score with 95% confidence interval.
ax[6] = plt.subplot(gs[2, 0:2])
ax_3 = sns.lineplot(data=df_reward, x="reward", y='Improvement Over Cen-MCTS (%)', hue='Algorithm', style='Algorithm', err_style='bars', errorbar='ci', err_kws={'elinewidth': 2, 'capsize': 8, 'capthick': 2}, linewidth=5, markersize=20, palette=color, markers=marker, ax=ax[6])
ax_3.set(xlabel="Number of Rewards \n (g)")

dir = "../../Data/full_comm"
algos = {"A": "A-MCTS", "Central": "Cen-MCTS", "Dec": "Dec-MCTS", "Global": "Global-MCTS", "Reset": "Reset-MCTS", "Greedy": "Greedy-MCTS"}
# Create dataframe action.
budget = [200, 250, 300, 350, 400]
df_action = create_dataframe(dir, "budget", budget, algos)
df_action = create_improvement_dataframe(df_action, 'budget', budget, algos)
# Plot the average score with 95% confidence interval.
ax[1] = plt.subplot(gs[0, 2:4])
ax_4 = sns.lineplot(data=df_action, x="budget", y='Improvement Over Cen-MCTS (%)', hue='Algorithm', style='Algorithm', err_style='bars', errorbar='ci', err_kws={'elinewidth': 2, 'capsize': 8, 'capthick': 2}, linewidth=5, markersize=20, palette=color, markers=marker, ax=ax[1])
ax_4.set(xlabel="Travel Budget (km) \n (b)")

# Create dataframe agent.
agent = ['20', '30', '40', '50', '60']
df_agent = create_dataframe(dir, "agent", agent, algos)
df_agent = create_improvement_dataframe(df_agent, 'agent', agent, algos)
# Plot the average score with 95% confidence interval.
ax[4] = plt.subplot(gs[1, 2:4])
ax_5 = sns.lineplot(data=df_agent, x="agent", y='Improvement Over Cen-MCTS (%)', hue='Algorithm', style='Algorithm', err_style='bars', errorbar='ci', err_kws={'elinewidth': 2, 'capsize': 8, 'capthick': 2}, linewidth=5, markersize=20, palette=color, markers=marker, ax=ax[4])
ax_5.set(xlabel="Number of Agents \n (e)")

# Create dataframe rewards.
reward = ['100', '200', '300', '400', '500']
df_reward = create_dataframe(dir, "reward", reward, algos)
df_reward = create_improvement_dataframe(df_reward, 'reward', reward, algos)
# Plot the average score with 95% confidence interval.
ax[7] = plt.subplot(gs[2, 2:4])
ax_6 = sns.lineplot(data=df_reward, x="reward", y='Improvement Over Cen-MCTS (%)', hue='Algorithm', style='Algorithm', err_style='bars', errorbar='ci', err_kws={'elinewidth': 2, 'capsize': 8, 'capthick': 2}, linewidth=5, markersize=20, palette=color, markers=marker, ax=ax[7])
ax_6.set(xlabel="Number of Rewards \n (h)")

dir = "../../Data/real_comm"
# Create dataframe action.
budget = [200, 250, 300, 350, 400]
df_action = create_dataframe(dir, "budget", budget, algos)
df_action = create_improvement_dataframe(df_action, 'budget', budget, algos)
# Plot the average score with 95% confidence interval.
ax[2] = plt.subplot(gs[0, 4:6])
ax_7 = sns.lineplot(data=df_action, x="budget", y='Improvement Over Cen-MCTS (%)', hue='Algorithm', style='Algorithm', err_style='bars', errorbar='ci', err_kws={'elinewidth': 2, 'capsize': 8, 'capthick': 2}, linewidth=5, markersize=20, palette=color, markers=marker, ax=ax[2])
ax_7.set(xlabel="Travel Budget (km) \n (c)")

# Create dataframe agent.
agent = ['20', '30', '40', '50', '60']
df_agent = create_dataframe(dir, "agent", agent, algos)
df_agent = create_improvement_dataframe(df_agent, 'agent', agent, algos)
# Plot the average score with 95% confidence interval.
ax[5] = plt.subplot(gs[1, 4:6])
ax_8 = sns.lineplot(data=df_agent, x="agent", y='Improvement Over Cen-MCTS (%)', hue='Algorithm', style='Algorithm', err_style='bars', errorbar='ci', err_kws={'elinewidth': 2, 'capsize': 8, 'capthick': 2}, linewidth=5, markersize=20, palette=color, markers=marker, ax=ax[5])
ax_8.set(xlabel="Number of Agents \n (f)")

# Create dataframe rewards.
reward = ['100', '200', '300', '400', '500']
df_reward = create_dataframe(dir, "reward", reward, algos)
df_reward = create_improvement_dataframe(df_reward, 'reward', reward, algos)
# Plot the average score with 95% confidence interval.
ax[8] = plt.subplot(gs[2, 4:6])
ax_9 = sns.lineplot(data=df_reward, x="reward", y='Improvement Over Cen-MCTS (%)', hue='Algorithm', style='Algorithm', err_style='bars', errorbar='ci', err_kws={'elinewidth': 2, 'capsize': 8, 'capthick': 2}, linewidth=5, markersize=20, palette=color, markers=marker, ax=ax[8])
ax_9.set(xlabel="Number of Rewards \n (i)")

f.subplots_adjust(wspace=.5)
for i in range(9):
    ax[i].set_ylim([-13, 35])
    ax[i].margins(x=0.02)
    ax[i].axhline(y=0, linewidth=3, color='orange', ls='--')
    try:
        ax[i].get_legend().remove()
    except:
        pass
handles, labels = ax[0].get_legend_handles_labels()
leg = f.legend(handles, labels, markerscale=1, fontsize=25, loc='upper center', bbox_to_anchor=(0.5, 0.93), ncols=5)
for line in leg.get_lines():
    line.set_linewidth(5.0)

# Save figure to pdf.
# plt.savefig("../../Figure/full_budget_agent_reward.pdf", format="pdf", bbox_inches="tight")

In [ ]:
from scipy import stats

# Function to calculate 95% confidence interval
def confidence_interval_95(column):
    mean = np.mean(column)
    n = len(column)
    std_err = stats.sem(column)  # Standard error of the mean
    h = std_err * stats.t.ppf((1 + 0.95) / 2, n - 1)  # Margin of error
    return mean, h

def create_table(df, var_name, vars, algos):
    table_mean = pd.DataFrame()
    table_conf = pd.DataFrame()
    for algo in algos:
        value_mean = dict()
        value_conf = dict()
        for var in vars:
            mean, h = confidence_interval_95(df[(df['Algorithm'] == algos[algo]) & (df[var_name] == var)]['IRC (%)'])
            value_mean[var] = "{:.2f}".format(mean)
            value_conf[var] = "{:.2f}".format(h)
        table_mean = pd.concat([table_mean, pd.DataFrame([value_mean], index=[algos[algo]])])
        table_conf = pd.concat([table_conf, pd.DataFrame([value_conf], index=[algos[algo]])])
    return table_mean, table_conf

# Variables.
intensity = [0.1, 0.3, 0.5, 0.7, 0.9]
planning = [100, 250, 500, 750, 1000]
comp = ['5', '10', '15', '20', '25']
action = ['7', '9', '11', '13', '15']
agent = ['10', '15', '20', '25', '30']
reward = ['100', '200', '300', '400', '500']
budget = [200, 250, 300, 350, 400]

# DC.
dir = "../../../INFOCOM-24/Data/9_action"
algos = {"RM": "A-MCTS", "Central": "Cen-MCTS", "Dec": "Dec-MCTS", "Global": "Dec-MCTS-Global", "Reset": "Dec-MCTS-Reset", "Greedy": "Greedy-MCTS"}

df_intensity = create_dataframe(dir, "intensity", intensity, algos)
table_1 = create_table(df_intensity, "intensity", intensity, algos)
df_planning = create_dataframe(dir, "planning", planning, algos)
table_2 = create_table(df_planning, "planning", planning, algos)
algos = {"RM": "A-MCTS", "Dec": "Dec-MCTS", "Global": "Dec-MCTS-Global", "Reset": "Dec-MCTS-Reset", "Greedy": "Greedy-MCTS"}
df_comp = create_dataframe(dir, "comp", comp, algos)
table_3 = create_table(df_comp, "comp", comp, algos)
algos = {"RM": "A-MCTS", "Central": "Cen-MCTS", "Dec": "Dec-MCTS", "Global": "Dec-MCTS-Global", "Reset": "Dec-MCTS-Reset", "Greedy": "Greedy-MCTS"}
df_action = create_dataframe(dir, "action", action, algos)
table_4 = create_table(df_action, "action", action, algos)
df_agent = create_dataframe(dir, "agent", agent, algos)
table_5 = create_table(df_agent, "agent", agent, algos)
df_reward = create_dataframe(dir, "reward", reward, algos)
table_6 = create_table(df_reward, "reward", reward, algos)
pd.concat([table_1[0], table_2[0], table_3[0], table_4[0], table_5[0], table_6[0]], axis=1).to_csv("../../Data/table/DC_mean.csv")
pd.concat([table_1[1], table_2[1], table_3[1], table_4[1], table_5[1], table_6[1]], axis=1).to_csv("../../Data/table/DC_conf.csv")

# ORI full comm.
planning = [40, 60, 80, 100, 120]
agent = ['20', '30', '40', '50', '60']

dir = "../../Data/full_comm"
algos = {"A": "A-MCTS", "Central": "Cen-MCTS", "Dec": "Dec-MCTS", "Global": "Dec-MCTS-Global", "Reset": "Dec-MCTS-Reset", "Greedy": "Greedy-MCTS"}

df_intensity = create_dataframe(dir, "intensity", intensity, algos)
table_1 = create_table(df_intensity, "intensity", intensity, algos)
df_planning = create_dataframe(dir, "planning", planning, algos)
table_2 = create_table(df_planning, "planning", planning, algos)
algos = {"A": "A-MCTS", "Dec": "Dec-MCTS", "Global": "Dec-MCTS-Global", "Reset": "Dec-MCTS-Reset", "Greedy": "Greedy-MCTS"}
df_comp = create_dataframe(dir, "comp", comp, algos)
table_3 = create_table(df_comp, "comp", comp, algos)
algos = {"A": "A-MCTS", "Central": "Cen-MCTS", "Dec": "Dec-MCTS", "Global": "Dec-MCTS-Global", "Reset": "Dec-MCTS-Reset", "Greedy": "Greedy-MCTS"}
df_action = create_dataframe(dir, "budget", budget, algos)
table_4 = create_table(df_action, "budget", budget, algos)
df_agent = create_dataframe(dir, "agent", agent, algos)
table_5 = create_table(df_agent, "agent", agent, algos)
df_reward = create_dataframe(dir, "reward", reward, algos)
table_6 = create_table(df_reward, "reward", reward, algos)
pd.concat([table_1[0], table_2[0], table_3[0], table_4[0], table_5[0], table_6[0]], axis=1).to_csv("../../Data/table/FC_mean.csv")
pd.concat([table_1[1], table_2[1], table_3[1], table_4[1], table_5[1], table_6[1]], axis=1).to_csv("../../Data/table/FC_conf.csv")

# ORI real comm.
dir = "../../Data/real_comm"

df_intensity = create_dataframe(dir, "intensity", intensity, algos)
table_1 = create_table(df_intensity, "intensity", intensity, algos)
df_planning = create_dataframe(dir, "planning", planning, algos)
table_2 = create_table(df_planning, "planning", planning, algos)
algos = {"A": "A-MCTS", "Dec": "Dec-MCTS", "Global": "Dec-MCTS-Global", "Reset": "Dec-MCTS-Reset", "Greedy": "Greedy-MCTS"}
df_comp = create_dataframe(dir, "comp", comp, algos)
table_3 = create_table(df_comp, "comp", comp, algos)
algos = {"A": "A-MCTS", "Central": "Cen-MCTS", "Dec": "Dec-MCTS", "Global": "Dec-MCTS-Global", "Reset": "Dec-MCTS-Reset", "Greedy": "Greedy-MCTS"}
df_action = create_dataframe(dir, "budget", budget, algos)
table_4 = create_table(df_action, "budget", budget, algos)
df_agent = create_dataframe(dir, "agent", agent, algos)
table_5 = create_table(df_agent, "agent", agent, algos)
df_reward = create_dataframe(dir, "reward", reward, algos)
table_6 = create_table(df_reward, "reward", reward, algos)
pd.concat([table_1[0], table_2[0], table_3[0], table_4[0], table_5[0], table_6[0]], axis=1).to_csv("../../Data/table/RC_mean.csv")
pd.concat([table_1[1], table_2[1], table_3[1], table_4[1], table_5[1], table_6[1]], axis=1).to_csv("../../Data/table/RC_conf.csv")